In [ ]:
import zipfile

In [ ]:
# MinIO connection details
minio_endpoint = os.getenv('AWS_S3_ENDPOINT')
access_key = os.getenv('AWS_ACCESS_KEY_ID')
secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
bucket_name = "models"
region = os.getenv('AWS_DEFAULT_REGION')

# Initialize the S3 client
s3_client = boto3.client(
    's3',
    endpoint_url=minio_endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    config=Config(signature_version='s3v4')
)

In [ ]:
with zipfile.ZipFile('replacement.zip', 'r') as zipf:
    zipf.extractall()

In [ ]:
with open('should_replace.json', 'r') as f:
    decision = json.load(f)

In [ ]:
def empty_production_folder:
    objects_to_delete = s3.list_objects_v2(Bucket=bucket_name, Prefix='production')

    if 'Contents' in objects_to_delete:
    delete_keys = {'Objects': []}
    for obj in objects_to_delete['Contents']:
        delete_keys['Objects'].append({'Key': obj['Key']})
    
    s3.delete_objects(Bucket=bucket_name, Delete=delete_keys)

In [ ]:
file_path = 'path/to/your/model.onnx' 

def upload_to_production_folder:
    s3.upload_file(file_path, bucket_name, folder_name + file_name_in_s3)

In [ ]:
if decision['replace']:
    empty_production_folder()
    upload_to_production_folder
else:
    print("Current model retained in production.")
